In [187]:
import glob
import os
import csv
import numpy as np
import pandas as pd

In [ ]:
# CAUTION: Need user input
metric = 'acc'
# metric = 'auc'

exp_prefixes = ["random", "entropy", "margin", "bald", "coreset"]
init_prefixes = ["hard", "easy", "gt_easy", "consistency", "uncertainty", "margin", "bald", "vaal", "coreset"]
# dataset_name = 'BLOODMNIST'
# dataset_name = 'ORGANAMNIST'
dataset_name = 'PATHMNIST'
output_path = './{}/'.format(dataset_name)
model = 'resnet18'
output_file_name = f'{dataset_name}_{model}_{metric}_ALL.csv'
init_num = '20'

In [ ]:
exp_list = []
for exp in exp_prefixes:
    for init in init_prefixes:
        exp_list.extend(glob.glob(os.path.join(output_path, model, '{}_{}_{}_*').format(init, init_num, exp)))

In [ ]:
exp_list[0]

In [ ]:
def exp_to_init_method(exp):
    exp = exp.split('/')[-1]
    exp = exp.split('_')[0]+'_'+exp.split('_')[1]

    if 'hard' in exp:
        return 'Hard-to-Contrast'
    elif 'easy' in exp and 'gt' not in exp:
        return 'Easy-to-Contrast'
    elif 'gt_easy' in exp:
        return 'Easy-to-Learn'
    elif 'consistency' in exp:
        return 'Consistency'
    elif 'uncertainty' in exp:
        return 'Entropy'
    elif 'margin' in exp:
        return 'Margin'
    elif 'bald' in exp:
        return 'BALD'
    elif 'vaal' in exp:
        return 'VAAL'
    elif 'coreset' in exp:
        return 'Coreset'
    else:
        return 'Random'

def exp_to_al_method(exp):
    exp = exp.split('_')[-2]

    if 'random' in exp:
        return 'Random'
    elif 'entropy' in exp:
        return 'Entropy'
    elif 'margin' in exp:
        return 'Margin'
    elif 'bald' in exp:
        return 'BALD'
    elif 'coreset' in exp:
        return 'Coreset'
    else:
        return 'Random'

In [ ]:
# columns = ['Method', 'Init_Method', 'AL_Method', '0', '1', '2', '3', '4', '5']
columns = ['Method', 'Init_Method', 'AL_Method', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
data = []

In [ ]:
for exp in exp_list:
    file_name = 'plot_episode_yvalues.txt' if metric is 'acc' else 'plot_episode_yvalues_auc.txt'
    file_path = os.path.join(exp, file_name)
    file = open(file_path, "r")
    values = file.readlines()
    values = [float(v[:-1]) for v in values]
    if metric is 'auc':
        values = [v * 100 for v in values]
    values.insert(0, exp_to_al_method(exp))
    values.insert(0, exp_to_init_method(exp))
    values.insert(0, f'{exp_to_init_method(exp)}_{exp_to_al_method(exp)}')
    data.append(values)

In [ ]:
df = pd.DataFrame(data, columns = columns)

In [ ]:
df

In [ ]:
df.to_csv(f'df_{output_file_name}', index=False)

In [ ]:
len(set(df.Method))

In [ ]:
data2 = []
for exp in exp_list:
    file_name = 'plot_episode_yvalues.txt' if metric is 'acc' else 'plot_episode_yvalues_auc.txt'
    file_path = os.path.join(exp, file_name)
    file = open(file_path, "r")
    values = file.readlines()
    values = [float(v[:-1]) for v in values]
    if metric is 'auc':
        values = [v * 100 for v in values]
    data_tuple = []
    method = f'{exp_to_init_method(exp)}_{exp_to_al_method(exp)}'
    for idx in range(len(values)):
        data_tuple.append([method, exp_to_init_method(exp), exp_to_al_method(exp), idx, values[idx]])
    data2.extend(data_tuple)

In [ ]:
df2 = pd.DataFrame(data2, columns=['Method', 'Init_Method', 'AL_Method', 'Episode', 'TestAccuracy'])
df2.Episode = df2.Episode.apply(lambda x: int((x+1)*10))

In [ ]:
df2.to_csv(f'df2_{output_file_name}', index=False)

In [ ]:
df2

In [ ]:
df2.to_csv(f'df_{output_file_name}', index=False)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

# df2 = pd.read_csv("VIKAS_CIFAR10.csv")

In [ ]:
len(set(df2.Method))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Arial']})
rc('text', usetex=True)

plt.rcParams["axes.labelweight"] = "normal"
plt.rcParams["font.weight"] = "normal"

sns.set_style("ticks")

In [ ]:
def export_legend(legend, filename="legend.png"):
    fig  = legend.figure
    fig.canvas.draw()
    bbox  = legend.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    fig.savefig(filename, dpi="figure", bbox_inches=bbox)

In [ ]:
len(set(df2.Init_Method))

In [ ]:
df2.Init_Method.value_counts()

In [ ]:
len(['black', 'red', 'springgreen',  'dodgerblue', 'peru', 'hotpink', 'blue', 'peru', 'purple'])

In [ ]:
df2

In [ ]:
al_methods = ['Random','Entropy','Margin','BALD', 'Coreset']
# display_methods= ['Random','Least Confidence (LC)','Max-Entropy (ME)','Min-Margin (MM)', 'ENS-varR','Coreset','VAAL', 'DBAL']
display_methods= al_methods
colors = ['blue', 'red', 'springgreen',  'dodgerblue', 'peru', 'hotpink', 'cyan', 'black', 'purple', 'grey']
colors = colors[:len(set(df2.Init_Method))]

for index in range(len(al_methods)):
    print(index)
#     if index != 6:
#         continue
    fig, ax = plt.subplots(figsize=(6,6))
    al_method = al_methods[index]
    axin2_ylims = {0:[90,91.5],1:[91,93], 2:[91,93], 3:[91,93], 4:[90,93], 5:[91,93], 6:[89,91.5], 7:[91,92]}
    axin_ylims = {0:[73.5,77],1:[74,77], 2:[73.2,77], 3:[73.5,77], 4:[71.9,76], 5:[72.5,77], 6:[73.5,77], 7:[72,76]}

    sns.set(rc={"lines.linewidth": 2})
    sns.set_style("ticks")

    sns.lineplot(x="Episode", y="TestAccuracy", hue="Init_Method", data=df2.loc[df2.AL_Method.isin([al_method]),:], \
                 ax=ax, palette=colors, linewidth = 3, legend=False)
    ax.set_xlabel('Images Labeled', size = 22, fontweight='bold')
    ylabel = 'Test Accuracy' if metric is 'acc' else 'Test AUC'
    ax.set_ylabel(ylabel, size = 22, fontweight='bold')

    ax.patch.set_edgecolor('black')  
    ax.patch.set_linewidth('2')

#     ax.set_xlabel('Labeled %', size = 20)
#     ax.set_ylabel('Test Accuracy', size = 20)
    ax.set_title(display_methods[index], size = 30)
    
    # ax.get_legend().set_title("Initial Pool Sampling Method")
    # plt.setp(ax.get_legend().get_texts(), fontsize='15') # for legend text
    # plt.setp(ax.get_legend().get_title(), fontsize='15') # for legend title
    
    # ax.set_yticklabels(ax.get_yticks(), size = 15, fontweight='bold')
    # ax.set_xticklabels(ax.get_xticks(), size = 15, fontweight='bold')

    x_labels = [str(t) for t in np.arange(20,130,10)]
    x_vals = np.arange(10,120,10)

    xticks = ax.set_xticks(x_vals)
    ax.set_xticklabels(x_labels)
    ax.set_xticklabels(x_labels, size = 20)
    # ax.set_xticklabels(ax.get_xticks(), size = 20)
    ax.set_yticklabels(ax.get_yticks(), size = 20)  

    # import pdb; pdb.set_trace()

#     

    # # COMMENT THIS BELOW PORTION AFTER RUNNING IT ONCE
    # ax.legend(loc='upper left', shadow=True, markerscale=1, ncol=1, prop={'size': 25})
    # leg = plt.legend(bbox_to_anchor=(6, 6), loc=2, borderaxespad=0., prop={'size': 25}, markerscale=2)
    # leg.get_frame().set_edgecolor('black')
    # export_legend(leg, "legend.pdf")

    # handles positioning of the zoomed image

    # axins = ax.inset_axes([0.27, 0.1, 0.3, 0.3])

    # # sub region of the original image
    # x1, x2, y1, y2 = 9, 12, axin_ylims[index][0], axin_ylims[index][1]
    # axins.set_xlim(x1, x2)
    # axins.set_ylim(y1, y2)
    # sns.lineplot(x="Episode", y="TestAccuracy", hue="Init_Method", data=df2.loc[df2.AL_Method.isin([al_method]),:], \
    #              ax=axins, legend=False, palette=colors, linewidth = 3)
    # axins.set_xlabel("")
    # axins.set_ylabel("")
    # axins.set_xticklabels('')
    # axins.set_yticklabels([str(round(float(label), 1)) for label in axins.get_yticks()])
    # # axins.set_yticklabels('')
    # axins.patch.set_edgecolor('black')  
    # axins.patch.set_linewidth('1') 
    # axins.tick_params(axis = "x", which = "both", bottom = False, top = False)
    # ax.indicate_inset_zoom(axins, edgecolor="black")

    # #################################################

    # axins2 = ax.inset_axes([0.68, 0.4, 0.3, 0.3])
    # # sub region of the original image
    # x1, x2, y1, y2 = 59, 61, axin2_ylims[index][0], axin2_ylims[index][1]
    # axins2.set_xlim(x1, x2)
    # axins2.set_ylim(y1, y2)
    # ax.indicate_inset_zoom(axins2, edgecolor="black")
    # sns.lineplot(x="Episode", y="TestAccuracy", hue="Init_Method", data=df2.loc[df2.AL_Method.isin([al_method]),:],\
    #              ax=axins2, legend=False, palette=colors, linewidth=3)
    # axins2.set_xlabel("")
    # axins2.set_ylabel("")
    # axins2.set_xticklabels('')
 
    # axins2.set_yticklabels([str(round(float(label), 1)) for label in axins2.get_yticks()])
    # # axins2.set_yticklabels('')
    # axins2.patch.set_edgecolor('black')  
    # axins2.patch.set_linewidth('1')
    # axins2.tick_params(axis = "x", which = "both", bottom = False, top = False)

    # ax.indicate_inset_zoom(axins2, edgecolor="black")


    plt.savefig(f'./figures/{dataset_name}_{al_methods[index]}_{metric}.pdf', dpi=500, format='pdf', bbox_inches = "tight")
    plt.savefig(f'./figures/pngs/{dataset_name}_{al_methods[index]}_{metric}.png', dpi=500, format='png', bbox_inches = "tight")
    
    plt.show()